In [167]:
pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import certifi
import json
from config import FMP_API_KEY

import warnings
warnings.filterwarnings("ignore")

# Data Collection

In [192]:
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

idx = pd.read_csv('data/index_symbols.csv')
comm = pd.read_csv('data/commodity_symbols.csv')

In [214]:
url = f"https://financialmodelingprep.com/stable/index-list?apikey={FMP_API_KEY}"
fmp_idx = pd.DataFrame(get_jsonparsed_data(url))
fmp_idx = fmp_idx[fmp_idx['symbol'].isin(idx['FMP API Symbol'])].reset_index(drop=True)

symbol = fmp_idx.loc[0, 'symbol']

url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from=1990-01-01&to=2025-09-26&apikey={FMP_API_KEY}"
df = pd.DataFrame(get_jsonparsed_data(url))[['symbol', 'date', 'close', 'volume']]

df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df.columns = pd.MultiIndex.from_product([[df['symbol'].iloc[0]], df.columns])
df = df.drop(columns=df.columns[0]).sort_index()

for i in range(1, len(fmp_idx)):
    symbol = fmp_idx.loc[i, 'symbol']
    url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from=1990-01-01&to=2025-09-26&apikey={FMP_API_KEY}"
    temp = pd.DataFrame(get_jsonparsed_data(url))[['symbol', 'date', 'close', 'volume']]
    temp['date'] = pd.to_datetime(temp['date'])
    temp = temp.set_index('date')
    temp.columns = pd.MultiIndex.from_product([[temp['symbol'].iloc[0]], temp.columns])
    temp = temp.drop(columns=temp.columns[0]).sort_index()
    df = pd.concat([df, temp], axis=1)

msci = pd.read_excel('data/MSCI_China_Index.xlsx')[:-1]
msci['Date'] = pd.to_datetime(msci['Date'])
# msci = msci.reindex(index=df.index)
msci = msci.set_index('Date')
msci.columns = pd.MultiIndex.from_product([['MSCI_China'], ['close']])
msci[('MSCI_China', 'volume')] = np.nan

df.join(msci, how='left').to_csv('data/index_data.csv')

In [215]:
url = f"https://financialmodelingprep.com/stable/commodities-list?apikey={FMP_API_KEY}"
fmp_comm = pd.DataFrame(get_jsonparsed_data(url))
fmp_comm = fmp_comm[fmp_comm['symbol'].isin(comm['FMP API Symbol'])].reset_index(drop=True)

symbol = fmp_comm.loc[0, 'symbol']

url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from=1990-01-01&to=2025-09-26&apikey={FMP_API_KEY}"
df = pd.DataFrame(get_jsonparsed_data(url))[['symbol', 'date', 'close', 'volume']]

df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df.columns = pd.MultiIndex.from_product([[df['symbol'].iloc[0]], df.columns])
df = df.drop(columns=df.columns[0]).sort_index()

for i in range(1, len(fmp_comm)):
    symbol = fmp_comm.loc[i, 'symbol']
    url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={symbol}&from=1990-01-01&to=2025-09-26&apikey={FMP_API_KEY}"
    temp = pd.DataFrame(get_jsonparsed_data(url))[['symbol', 'date', 'close', 'volume']]
    temp['date'] = pd.to_datetime(temp['date'])
    temp = temp.set_index('date')
    temp.columns = pd.MultiIndex.from_product([[temp['symbol'].iloc[0]], temp.columns])
    temp = temp.drop(columns=temp.columns[0]).sort_index()
    df = pd.concat([df, temp], axis=1)

# df.to_csv('data/commodity_data.csv')
nickel = pd.read_csv('data/Nickel_futures.csv', parse_dates=['Date'], dayfirst=True, index_col='Date')\
    .rename_axis('date').sort_index().rename(columns={'Price': 'close'})[['close']]
nickel.columns = pd.MultiIndex.from_product([['Nickel'], nickel.columns])
nickel[('Nickel', 'volume')] = np.nan
df.join(nickel, how='left').to_csv('data/commodity_data.csv')

# Data Preprocessing

In [188]:
print("MSCI_China is in USD")
fmp_idx

MSCI_China is in USD


,symbol,name,exchange,currency
0,^GSPTSE,S&P/TSX Composite Index,TSX,CAD
1,^TWII,TSEC Weighted Index,TAI,TWD
2,^AXJO,S&P/ASX 200,ASX,AUD
3,^GSPC,S&P 500,SNP,USD
4,^N225,Nikkei 225,OSA,JPY
5,^KS11,KOSPI Composite Index,KSC,KRW
6,FTSEMIB.MI,FTSE MIB Index,MIL,EUR
7,^FTSE,FTSE 100,FGI,GBP
8,^SSMI,SMI PR,SIX,CHF
9,^NSEI,NIFTY 50,NSE,INR


In [ ]:
# TODO: convert all to USD 
equity_index_data = pd.read_csv('data/index_data.csv', index_col=0, header=[0,1])
equity_index_data.tail()

^GSPTSE                  ^TWII              ^AXJO            \
              close       volume     close     volume   close    volume   
2025-09-22  29959.0  352957619.0  25880.60  4316700.0  8810.9  611200.0   
2025-09-23  29815.6  296641424.0  26247.37  4775000.0  8845.9  742600.0   
2025-09-24  29757.0  312079721.0  26196.73  4309500.0  8764.5  715300.0   
2025-09-25  29732.0  293274800.0  26023.85  5318500.0  8773.0  837400.0   
2025-09-26  29761.3  297599800.0  25580.32  4370400.0  8787.7  832800.0   

              ^GSPC                   ^N225               ...     ^NSEI  \
              close        volume     close       volume  ...     close   
2025-09-22  6693.74  5.642620e+09  45493.66  116600000.0  ...  25202.35   
2025-09-23  6656.93  5.633620e+09       NaN          NaN  ...  25169.50   
2025-09-24  6637.98  5.459180e+09  45630.31  131700000.0  ...  25056.90   
2025-09-25  6604.73  5.874670e+09  45754.93  127200000.0  ...  24890.85   
2025-09-26  6643.71  5.103110e+09  45354.99  147500000.0  ...  24654.70   

                             ^NDX                  ^GDAXI              \
                 volume     close        volume     close      volume   
2025-09-22  254509892.0  24761.07  1.429210e+09  23527.05  53681300.0   
2025-09-23  299204338.0  24580.17  1.294683e+09  23611.33  48153500.0   
2025-09-24  244382281.0  24503.57  1.223742e+09  23666.81  54163200.0   
2025-09-25  342534768.0  24397.31  1.435051e+09  23534.83  55308500.0   
2025-09-26  291537949.0  24503.85  1.235069e+09  23739.47  45110800.0   

              ^FCHI              MSCI_China         
              close      volume       close volume  
2025-09-22  7830.11  42599900.0  190.769091    NaN  
2025-09-23  7872.02  48991400.0  189.082017    NaN  
2025-09-24  7827.45  52936900.0  192.635829    NaN  
2025-09-25  7795.42  51878400.0  192.769367    NaN  
2025-09-26  7870.68  52920700.0  189.959098    NaN  

[5 rows x 28 columns]

In [ ]:
# All commodities are in USD
commodity_data = pd.read_csv('data/commodity_data.csv', index_col=0, header=[0,1])
commodity_data.tail()

ALIUSD           GCUSD             KEUSX            ZCUSX  \
              close  volume   close    volume   close   volume   close   
date                                                                     
2025-09-22  2530.25  2612.0  3775.1  219170.0  502.25  31994.0  421.75   
2025-09-23  2522.00    44.0  3815.7  291564.0  511.50  31994.0  426.25   
2025-09-24  2533.50  2113.0  3768.1  236560.0  506.75  24625.0  424.25   
2025-09-25  2551.00     1.0  3771.1  262183.0  512.25  20671.0  425.75   
2025-09-26  2544.75  1914.0  3809.0  214083.0  505.50  20671.0  422.00   

                       HGUSD           ...    LEUSX           NGUSD            \
              volume   close   volume  ...    close   volume  close    volume   
date                                   ...                                      
2025-09-22  189769.0  4.6320  22730.0  ...  242.350  42757.0  2.806  163184.0   
2025-09-23  189769.0  4.6440  23732.0  ...  241.125  26537.0  2.853  163184.0   
2025-09-24  145931.0  4.8135  72671.0  ...  238.975  23186.0  2.858   70892.0   
2025-09-25  146851.0  4.7580  65306.0  ...  236.050  28829.0  2.904   74214.0   
2025-09-26  146851.0  4.7715  35205.0  ...  236.500  18874.0  3.206  168752.0   

             KCUSX           BZUSD              Nickel         
             close   volume  close   volume      close volume  
date                                                           
2025-09-22  367.35  25299.0  66.57  24400.0  15,197.13    NaN  
2025-09-23  350.15  25338.0  67.63  27893.0  15,335.38    NaN  
2025-09-24  367.75  25338.0  69.31  28459.0  15,413.13    NaN  
2025-09-25  371.35  18066.0  69.42  28459.0  15,252.88    NaN  
2025-09-26  378.05  14512.0  70.13  21388.0  15,150.38    NaN  

[5 rows x 24 columns]